In [2]:
import nltk
nltk.download('reuters')

[nltk_data] Downloading package reuters to /root/nltk_data...


True

In [3]:
import nltk
import random
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict


tokens = reuters.words()

bi_tokens = list(bigrams(tokens))
tri_tokens = list(trigrams(tokens))

bi_freq = nltk.FreqDist(bi_tokens)
tri_freq = nltk.FreqDist(tri_tokens)

tri_model = defaultdict(lambda: defaultdict(lambda: 0))

for w1, w2, w3 in tri_tokens:
  tri_model[(w1, w2)][w3] += 1

def generate_text(seed_word, num_words):
  text = [seed_word[0], seed_word[1]]
  for i in range(num_words):
    next_word = random.choice(list(tri_model[(text[-2], text[-1])].keys()))
    text.append(next_word)
  return ' '.join(text)

def evaluate_model(seed_word, num_sents, num_words_per_sent):
  print(f"Generating {num_sents} sentences of {num_words_per_sent} words each")
  for i in range(num_sents):
    generated_sent = generate_text(seed_word, num_words_per_sent)
    print(f"Sentence {i+1}: {generated_sent}")


seed_word = ("today", "on")
num_sents = 5
num_word_per_sent = 10

evaluate_model(seed_word, num_sents, num_word_per_sent)

Generating 5 sentences of 10 words each
Sentence 1: today on the heavy rains but it should realized 10 , Bank
Sentence 2: today on a capital reorganization to allow you to evaluate a number
Sentence 3: today on weather - related topic on the troubled international coffee prices
Sentence 4: today on natural rubber over the enlargement of the Gallaher group .
Sentence 5: today on its behalf 2 . 40 Exports 7 . 36 PCT


In [4]:
from nltk.util import ngrams

quad_tokens = list(ngrams(tokens, 4))

quad_freq = nltk.FreqDist(quad_tokens)

quad_model = defaultdict(lambda: defaultdict(lambda: 0))

for w1, w2, w3, w4 in quad_tokens:
  quad_model[(w1, w2, w3)][w4] += 1


def generate_text(seed_word, num_words):
  text = list(seed_word)
  for i in range(num_words):
    next_word = random.choice(list(quad_model[(text[-3], text[-2], text[-1])].keys()))
    text.append(next_word)
  return ' '.join(text)

seed_word = ("today", "on", "the")
num_sents = 5
num_word_per_sent = 15

evaluate_model(seed_word, num_sents, num_word_per_sent)

Generating 5 sentences of 15 words each
Sentence 1: today on the main sticking point on quotas has been Brazil ' s 20 mln dlrs versis 40
Sentence 2: today on the NYSE at 30 - 1 / 4 in active trading . Vague rumors that Wendy
Sentence 3: today on the NYSE . Reed outlined a plan to limit total exports to 90 , 000 tonnes
Sentence 4: today on the issue continues ," DiBona said at a ceremenony to sign the extension of credit by
Sentence 5: today on the issue will be followed by a bond of around one billion U . S later


In [ ]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

text = """The quick brown fox jumps over the lazy dog. A quick brown dog outpaces a quick fox. The dog and the fox like to run in the forest. However, the lazy dog prefers to sleep all day."""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100, verbose=1)

# Function to generate text
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


# Generate text
seed_text = "fox"
next_words = 5
generated_text = generate_text(seed_text, next_words, model, max_sequence_len)
print(generated_text)


Epoch 1/100
2/2 [==============================] - 5s 70ms/step - loss: 3.0910 - accuracy: 0.0278
Epoch 2/100
2/2 [==============================] - 0s 67ms/step - loss: 3.0721 - accuracy: 0.1389
Epoch 3/100
2/2 [==============================] - 0s 58ms/step - loss: 3.0515 - accuracy: 0.0833
Epoch 4/100
2/2 [==============================] - 0s 61ms/step - loss: 3.0115 - accuracy: 0.0833
Epoch 5/100
2/2 [==============================] - 0s 66ms/step - loss: 2.9042 - accuracy: 0.1111
Epoch 6/100
2/2 [==============================] - 0s 64ms/step - loss: 2.8688 - accuracy: 0.1111
Epoch 7/100
2/2 [==============================] - 0s 78ms/step - loss: 3.0075 - accuracy: 0.1111
Epoch 8/100
2/2 [==============================] - 0s 63ms/step - loss: 2.9057 - accuracy: 0.1111
Epoch 9/100
2/2 [==============================] - 0s 69ms/step - loss: 2.7966 - accuracy: 0.1389
Epoch 10/100
2/2 [==============================] - 0s 71ms/step - loss: 2.7500 - accuracy: 0.1667
Epoch 11/100
2/2 [=